# Setup Instructions

Please run the following cell to install the required packages.  

- After installation, a **runtime restart** is required because Colab’s default packages (e.g., NumPy) are newer than those required by CytoTRACE2, which may lead to compatibility issues.
  - Please click **Runtime → Restart session and run all** to execute the notebook from the top.  
The installation cell will run briefly again. This is expected and completely fine.

In [1]:
!pip install \
  cytotrace2-py==1.1.0.3 \
  seaborn==0.13.2 \
  scanpy==1.11.2 \
  anndata==0.11.4

# **To run the entire notebook:** click Runtime → Run all

In [2]:
import warnings
import logging
import matplotlib
from numba.core.errors import NumbaWarning

# suppress the Helvetica font
warnings.filterwarnings("ignore", message="findfont: Font family")
# Suppress font-related messages
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

# Suppress the deprecation warning from cytotrace2_py
warnings.filterwarnings(
    "ignore",
    message="pkg_resources is deprecated as an API.*",
    category=UserWarning,
    module="cytotrace2_py.common.gen_utils"
)
# Suppress numba warning
warnings.filterwarnings("ignore", category=NumbaWarning)

In [3]:
import gdown
import numpy
import os
import requests
import pandas as pd
from cytotrace2_py.cytotrace2_py import cytotrace2
import matplotlib.pyplot as plt

In [4]:
os.makedirs("data", exist_ok=True)

In [5]:
outputs = ["data/Cord_blood_CITE_seq_downsampled_expression.txt", "data/Cord_blood_CITE_seq_downsampled_annotation.txt"]
file_ids = ["1EsUXJbqZCx143ELBplC5gOZpQPPfuGXM", "1zAA69hlLTMs1h2UohEAImzJH0eZIdLFg"]
for i, f_id in enumerate(file_ids):
    url = f"https://drive.google.com/uc?id={f_id}"
    # Download only if the file doesn't already exist
    if not os.path.exists(outputs[i]):
        gdown.download(url, outputs[i], quiet=False)
    else:
        print("Dataset already downloaded.")

Downloading...
From: https://drive.google.com/uc?id=1EsUXJbqZCx143ELBplC5gOZpQPPfuGXM
To: /content/data/Cord_blood_CITE_seq_downsampled_expression.txt
100%|██████████| 94.5M/94.5M [00:00<00:00, 271MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zAA69hlLTMs1h2UohEAImzJH0eZIdLFg
To: /content/data/Cord_blood_CITE_seq_downsampled_annotation.txt
100%|██████████| 59.0k/59.0k [00:00<00:00, 54.5MB/s]


In [6]:
# Load expression matrix (genes × cells)
expression = pd.read_csv("data/Cord_blood_CITE_seq_downsampled_expression.txt", sep="\t", index_col=0)
print("Expression shape:", expression.shape)
expression.head()

Expression shape: (20401, 2308)


,TTTATGCGTCCGTTAA,GCATGATCACAACGTT,GGTATTGTCCAGTATG,CGCTTCACAGTGGAGT,TCCCGATTCCTCATTA,CTAGTGATCGAATGGG,GTCACGGCATGTTCCC,TTGTAGGGTGTTGGGA,ATGCGATTCGGTGTTA,AAACGGGCAATCCAAC,...,CATCAGAGTTACAGAA,TTAGGACAGACGACGT,TATGCCCTCTTCTGGC,CTCGGAGCAACACCCG,GCTCCTACACTGAAGG,AGATTGCGTTAGGGTG,AACTTTCTCGACAGCC,TGACAACCAAGCCATT,TTTCCTCGTACCGCTG,TGCCCATAGGATGGAA
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1BG-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Load annotation matrix (cells x phenotype)
annotation = pd.read_csv("data/Cord_blood_CITE_seq_downsampled_annotation.txt", sep="\t", index_col=0)
print("Annotation shape:", annotation.shape)
annotation.head()

Annotation shape: (2308, 1)


,phenotype
TTTATGCGTCCGTTAA,B cell
GCATGATCACAACGTT,B cell
GGTATTGTCCAGTATG,B cell
CGCTTCACAGTGGAGT,B cell
TCCCGATTCCTCATTA,B cell


In [ ]:
results = cytotrace2("data/Cord_blood_CITE_seq_downsampled_expression.txt", annotation_path="data/Cord_blood_CITE_seq_downsampled_annotation.txt", species="human")

cytotrace2: Input parameters
    Input file: data/Cord_blood_CITE_seq_downsampled_expression.txt
    Species: human
    Parallelization enabled: True
    Batch size: 20000
    Smoothing batch size: 1000
    Seed: 14
    Output directory: cytotrace2_results
    Plotting enabled: True
    Verbose mode enabled: True
    User-provided limit for number of cores to use: None
       ...2 cores detected. CytoTRACE 2 will run using up to 1/2 cores.
cytotrace2: Loading dataset
cytotrace2: Dataset characteristics
    Number of input genes:  20401
    Number of input cells:  2308
cytotrace2: Checking models...


Downloading...
From: https://drive.google.com/uc?export=download&id=1nyZeMTn3oSCO0bKrR0Pynu7cr6xfs3yY
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model1.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 178MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1x7JYdmiyKG7y_7U37prGTTqtkKGqyZND
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model2.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 272MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1vOjvGWa2Vb8QEfKkKYMTaNiS3e0UjjMo
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model3.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 248MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1IqMXiKf3Td_h251JO3mhj9pu4FhnH4Le
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model4.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 212MB/s]


...

Downloading...
From: https://drive.google.com/uc?export=download&id=1nNLwCvGrsJ0iYflaysh6YWIKanPaxG-r
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model14.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 273MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1dGSjNn3-YAA390Z-wCP5pCWBRo1aOs50
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model15.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 243MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1QovpMD_tdGOszSDx8qEFrNLpq4PFnQtp
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model16.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 238MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1ZMgukaZNzVAkyQCvYvAQJ53qk2RwsW-i
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model17.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 130MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1yUC9ndmr6t3nLzIVc8LRIxT4qV-kVrVg
To: /usr/local/lib/python3.11/dist-packages/cytotrace2_py/resources/models/model18.pt
100%|██████████| 8.24M/8.24M [00:00<00:00, 111MB/s]
